In [12]:
from PIL import Image
import numpy as np

def remove_white_background(input_path, output_path, threshold=240):
    """
    Remove white background from PNG image and make it transparent.
    
    Args:
        input_path (str): Path to input PNG file
        output_path (str): Path to save output PNG file
        threshold (int): White threshold value (0-255). Pixels above this value will be made transparent.
    """
    try:
        # Open the image
        img = Image.open(input_path)
        
        # Convert to RGBA if not already (to support transparency)
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        
        # Convert image to numpy array for easier manipulation
        data = np.array(img)
        
        # Create a mask for white pixels
        # A pixel is considered "white" if all RGB values are above the threshold
        white_mask = (data[:, :, 0] >= threshold) & (data[:, :, 1] >= threshold) & (data[:, :, 2] >= threshold)
        
        # Set alpha channel to 0 (transparent) for white pixels
        data[white_mask, 3] = 0
        
        # Convert back to PIL Image
        result_img = Image.fromarray(data, 'RGBA')
        
        # Save the result
        result_img.save(output_path, 'PNG')
        
        print(f"Background removed successfully! Saved to: {output_path}")
        
    except FileNotFoundError:
        print(f"Error: File '{input_path}' not found.")
    except Exception as e:
        print(f"Error processing image: {e}")

def remove_white_background_advanced(input_path, output_path, tolerance=30):
    """
    Advanced version that handles near-white colors and anti-aliasing better.
    
    Args:
        input_path (str): Path to input PNG file
        output_path (str): Path to save output PNG file
        tolerance (int): How close to white a pixel needs to be to become transparent (0-100)
    """
    try:
        # Open the image
        img = Image.open(input_path)
        
        # Convert to RGBA if not already
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        
        # Convert to numpy array
        data = np.array(img)
        
        # Calculate how "white" each pixel is
        # Use the minimum of RGB values - white pixels have high values in all channels
        min_rgb = np.min(data[:, :, :3], axis=2)
        max_rgb = np.max(data[:, :, :3], axis=2)
        
        # For truly white pixels, min and max RGB should be close and high
        whiteness = np.minimum(min_rgb, 255 - (max_rgb - min_rgb))
        
        # Create transparency based on whiteness
        # Map whiteness to alpha: very white = transparent, not white = opaque
        white_threshold = 255 - tolerance
        alpha_new = np.where(whiteness >= white_threshold, 
                           np.maximum(0, 255 - (whiteness - white_threshold) * 255 // tolerance),
                           255)
        
        # Apply the new alpha values
        data[:, :, 3] = np.minimum(data[:, :, 3], alpha_new)
        
        # Convert back to PIL Image
        result_img = Image.fromarray(data, 'RGBA')
        
        # Save the result
        result_img.save(output_path, 'PNG')
        
        print(f"Background removed successfully! Saved to: {output_path}")
        
    except FileNotFoundError:
        print(f"Error: File '{input_path}' not found.")
    except Exception as e:
        print(f"Error processing image: {e}")

def remove_black_background(input_path, output_path, threshold=30):
    """
    Remove black background from PNG image and make it transparent.
    
    Args:
        input_path (str): Path to input PNG file
        output_path (str): Path to save output PNG file
        threshold (int): Black threshold value (0-255). 
                         Pixels with all RGB values below this are made transparent.
    """
    try:
        # Open the image
        img = Image.open(input_path)
        
        # Convert to RGBA to support transparency
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        
        # Convert to numpy array
        data = np.array(img)
        
        # Create a mask for black (or near-black) pixels
        black_mask = (data[:, :, 0] <= threshold) & \
                     (data[:, :, 1] <= threshold) & \
                     (data[:, :, 2] <= threshold)
        
        # Set alpha channel to 0 for black pixels
        data[black_mask, 3] = 0
        
        # Convert back to PIL image
        result_img = Image.fromarray(data, 'RGBA')
        
        # Save output
        result_img.save(output_path, 'PNG')
        
        print(f"Black background removed successfully! Saved to: {output_path}")
        
    except FileNotFoundError:
        print(f"Error: File '{input_path}' not found.")
    except Exception as e:
        print(f"Error processing image: {e}")

In [ ]:
input_file = "icon.png"
output_file = "icon.png"

# Method 1: Simple white removal (good for solid white backgrounds)
# remove_white_background(input_file, output_file, threshold=240)

# Method 2: Advanced removal (better for images with anti-aliasing or near-white colors)
remove_white_background_advanced(input_file, "output_advanced.png", tolerance=30)

# Example usage:
# remove_black_background(input_file, output_file, threshold=40)


Background removed successfully! Saved to: output_advanced.png


/var/folders/f2/lbj2f9h979b75rj33ps9wq780000gn/T/ipykernel_27776/2861213874.py:83: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  result_img = Image.fromarray(data, 'RGBA')


In [ ]:
def png_to_jpg(input_path, output_path, bg_color=(255, 255, 255)):
    """
    Convert a PNG image to JPG format.
    If the PNG has transparency, the transparent areas will be filled with bg_color.
    
    Args:
        input_path (str): Path to input PNG file
        output_path (str): Path to save output JPG file
        bg_color (tuple): RGB background color to replace transparency (default: white)
    """
    img = Image.open(input_path).convert("RGBA")
    
    # Create a background image
    background = Image.new("RGB", img.size, bg_color)
    
    # Paste PNG on top of background using alpha channel as mask
    background.paste(img, mask=img.split()[3])
    
    # Save as JPG
    background.save(output_path, "JPEG")
    print(f"Converted PNG to JPG successfully! Saved to: {output_path}")

Converted PNG to JPG successfully! Saved to: output.jpg


In [ ]:
# Example usage:
png_to_jpg(input_file, "output.jpg", bg_color=(0, 0, 0))